<a href="https://colab.research.google.com/github/semhoun/omnius/blob/main/nb/FrenchSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>


# Model List
[Model lists](https://huggingface.co/unsloth):
 - unsloth/Meta-Llama-3.1-8B-bnb-4bit ***Llama-3.1 15 trillion tokens model 2x faster!***
 - unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
 - unsloth/Meta-Llama-3.1-70B-bnb-4bit
 - unsloth/Meta-Llama-3.1-405B-bnb-4bit ***We also uploaded 4bit for 405b!***
 - unsloth/Mistral-Nemo-Base-2407-bnb-4bit ***New Mistral 12b 2x faster!***
 - unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
 - unsloth/mistral-7b-v0.3-bnb-4bit ***Mistral v3 2x faster!***
 - unsloth/mistral-7b-instruct-v0.3-bnb-4bit
 - unsloth/Phi-3.5-mini-instruct ***Phi-3.5 2x faster!***
 - unsloth/Phi-3-medium-4k-instruct
 - unsloth/gemma-2-9b-bnb-4bit
 - unsloth/gemma-2-27b-bnb-4bit ***Gemma 2x faster!***
 - unsloth/SmolLM3-3B-bnb-4bit

Remember to go to https://huggingface.co/settings/tokens for a token!


# Definitions


In [1]:
from google.colab import userdata

model_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

#source_model = "mistralai/Mistral-Nemo-Instruct-2407"
#source_model = "mistralai/Mistral-7B-Instruct-v0.3"
#source_model = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
source_model = "HuggingFaceTB/SmolLM3-3B"
#model_name = "Claire-12B-v1.0"
model_name = "Claire-Dev"
hf_username = "nsemhoun"
hf_token = userdata.get('HuggingFaceToken')

save_local = False
save_hf = True

# Installation

In [2]:
%%capture
import os, re
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3"
!pip install --upgrade unsloth-zoo
!pip install --upgrade unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

# Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = source_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = model_4bit,
    token = hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.4: Fast Smollm3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.629 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM3-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


### Initial model test
We try some chat to compare with the trained model

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "human", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>system\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\nThe next numbers in the Fibonacci sequence would be 13, 21, 34, 55, 89, and so on. The sequence is formed by adding the two preceding numbers to get the next number. So, after 8, the next number is 13 (5 + ']

#Dataset Preparation and Processing
We now use the `ChatML` format for conversation style finetunes. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style if needed
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    texts = []
    if (examples.get('system') == None):
      transformed_data = [
          {"role": "user", "content": examples.get('question')},
          {"role": "assistant", "content": examples.get('chosen')},
      ]
    else:
      transformed_data = [
          {"role": "system", "content": examples.get('system')},
          {"role": "user", "content": examples.get('question')},
          {"role": "assistant", "content": examples.get('chosen')},
      ]
    text = tokenizer.apply_chat_template(transformed_data, tokenize = False, add_generation_prompt = False)
    return { "text" : text, }
pass

from datasets import load_dataset
dataset = load_dataset("jpacifico/french-orca-dpo-pairs-revised", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = False,)

README.md:   0%|          | 0.00/676 [00:00<?, ?B/s]

french_orca_rlhf_revised.jsonl:   0%|          | 0.00/44.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12670 [00:00<?, ? examples/s]

Map:   0%|          | 0/12670 [00:00<?, ? examples/s]

Let's see how the format works by printing the 5th element

In [6]:
import pprint
row = dataset[5]
print('SYSTEM: ' + '=' * 50)
pprint.pprint(row["system"])
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["question"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('CHAT TEMPLATE: ' + '=' * 50)
pprint.pprint(row["text"])


SYSTEM: ==================================================
'Vous êtes un assistant IA qui aide les gens à trouver des informations.'
INSTRUCTION: ==================================================
('Compte tenu du principe du flux de conscience, proposez une question et une '
 'réponse pertinentes. Justification : Dans ce contexte, le jeu fait référence '
 'à une partie de volleyball où une joueuse de beach-volley effectue le '
 'service.\n'
 'La question et la réponse :')
ACCEPTED: ==================================================
("Question\xa0: Quelle est la technique appropriée pour qu'une joueuse de "
 'beach-volley puisse servir le ballon efficacement dans un match\xa0?\n'
 '\n'
 'Réponse : Pour servir le ballon efficacement au beach-volley, une joueuse '
 'doit adopter une routine de pré-service cohérente, se tenir dans une '
 'position équilibrée avec les pieds écartés à la largeur des épaules, lancer '
 'le ballon à une hauteur appropriée tout en étendant son bras non dominan

# Model Training Setup


Add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
lora_rank = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


Setup the SFT traincer with approprite arguments.

*We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.*

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, #1
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1, #2
        gradient_accumulation_steps = 4, #4
        warmup_steps = 5,
        #max_steps = 60,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # remove to activate WandDB
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/12670 [00:00<?, ? examples/s]

# Training Execution
Execute the training process with the configured trainer and monitor the training progress.

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.629 GB.
5.863 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,670 | Num Epochs = 1 | Total steps = 3,168
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 30,228,480 of 3,105,327,104 (0.97% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.606700
2,1.417100
3,1.435800
4,1.523300
5,1.354600
6,1.281500
7,1.488200
8,1.588400
9,1.662300
10,1.428000


In [13]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6857.7929 seconds used for training.
114.3 minutes used for training.
Peak reserved memory = 6.645 GB.
Peak reserved memory for training = 0.782 GB.
Peak reserved memory % of max memory = 57.142 %.
Peak reserved memory for training % of max memory = 6.725 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [18]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|im_start|>user\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\nPour continuer la suite de la séquence de Fibonacci, on ajoute le dernier nombre (5) au nombre précédent (3), ce qui donne 8. La suite de Fibonacci est : 1, 1, 2, 3, 5, 8, 13.<|im_end|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>system
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [34]:
if False:
  model.save_pretrained("lora_" + model_name)  # Local saving
  tokenizer.save_pretrained("lora_" + model_name)
  # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
  # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [20]:
messages = [
    {"role": "user", "content": "Quel est la fameuse grande tour à Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
Quel est la fameuse grande tour à Paris?<|im_end|>
<|im_start|>assistant
La tour Eiffel est une structure emblématique située à Paris, en France. Construite en 1889 pour l'Exposition universelle de Paris, elle est devenue un symbole emblématique de la ville et de la France.<|im_end|>


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [21]:
if model_4bit:
  # Merge to 4bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_4bit",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_4bit", token = hf_token)
else:
  # Merge to 16bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_16bit", token = hf_token)

if False:
  # Just LoRA adapters
  if save_local:
    model.save_pretrained(model_name, save_method = "lora",)
    tokenizer.save_pretrained(model_name, save_method = "lora",)
  if save_hf:
    model.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)
    tokenizer.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...
Successfully copied all 2 files from cache to Claire-Dev.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [03:56<00:00, 118.07s/it]


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [22]:
# Save to local
if save_local:
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q8_0")
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "f16")
  # model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q4_k_m")


# Save to multiple GGUF options - much faster if you want multiple!
if save_hf:
    model.push_to_hub_gguf(
        hf_username + "/" + model_name + "-GGUF",
        tokenizer,
        #quantization_method = ["q4_k_m", "q8_0", "q4_0", "f16"],
        quantization_method = ["q8_0", "f16"],
        token = hf_token,
    )

Unsloth: Updating system package directories
Unsloth: Install GGUF and other packages
Unsloth GGUF:hf-to-gguf:Loading model: Claire-Dev
Unsloth GGUF:hf-to-gguf:Model architecture: SmolLM3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {2048, 128256}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,          torch.bfloat16 --> F32, shape = {2048}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:gguf: context length = 65536
Unsloth GGUF:hf-to-gguf:gguf: embedding length = 2048
Unsloth GGUF:hf-to-gguf:gguf: feed for

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to Claire-Dev.Q8_0.gguf with size = 3.3G
Unsloth: Successfully saved GGUF to:
Claire-Dev.Q8_0.gguf
Unsloth GGUF:hf-to-gguf:Loading model: Claire-Dev
Unsloth GGUF:hf-to-gguf:Model architecture: SmolLM3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {2048, 128256}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,          torch.bfloat16 --> F32, shape = {2048}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:gguf: context length = 65536
Uns

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to Claire-Dev.F16.gguf with size = 6.2G
Unsloth: Successfully saved GGUF to:
Claire-Dev.F16.gguf


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
